# Train a Custom Object Detection Model with TensorFlow Lite Model Maker

In this colab notebook, you'll learn how to use the [TensorFlow Lite Model Maker](https://www.tensorflow.org/lite/guide/model_maker) to train a custom object detection model to detect Android figurines and how to put the model on a Raspberry Pi.

The Model Maker library uses *transfer learning* to simplify the process of training a TensorFlow Lite model using a custom dataset. Retraining a TensorFlow Lite model with your own custom dataset reduces the amount of training data required and will shorten the training time.


## Preparation

### Install the required packages
Start by installing the required packages, including the Model Maker package from the [GitHub repo](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker) and the pycocotools library you'll use for evaluation.

In [1]:
!pip install tensorflow==2.7.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     | 665.5 MB 12.2 MB/s
     |████████████████████████████████| 463 kB 5.1 MB/s 
     |████████████████████████████████| 1.3 MB 62.2 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.2+zzzcolab20220527125636
    Uninstalling tensorflow-2.8.2+zzzcolab20220527125636:
      Successfully uninstalled tensorflow-2.8.2+zzzcolab20220527125636


In [2]:
!pip install -q tflite-model-maker
!pip install -q tflite-support
!sudo apt-get install libportaudio2

     |████████████████████████████████| 642 kB 4.9 MB/s 
     |████████████████████████████████| 42.5 MB 2.3 MB/s 
     |████████████████████████████████| 87 kB 8.4 MB/s 
     |████████████████████████████████| 596 kB 63.0 MB/s 
     |████████████████████████████████| 840 kB 66.8 MB/s 
     |████████████████████████████████| 11.2 MB 20.9 MB/s 
     |████████████████████████████████| 237 kB 74.7 MB/s 
     |████████████████████████████████| 1.2 MB 56.3 MB/s 
     |████████████████████████████████| 120 kB 68.0 MB/s 
     |████████████████████████████████| 77 kB 7.7 MB/s 
     |████████████████████████████████| 1.1 MB 60.4 MB/s 
     |████████████████████████████████| 3.4 MB 30.3 MB/s 
     |████████████████████████████████| 6.4 MB 48.1 MB/s 
     |████████████████████████████████| 25.3 MB 98.9 MB/s 
     |████████████████████████████████| 99 kB 12.0 MB/s 
     |████████████████████████████████| 48.3 MB 1.9 MB/s 
     |████████████████████████████████| 352 kB 76.1 MB/s 
     |████████████

Import the required packages.

In [3]:
!pip install scann==1.2.6

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.9 MB 5.5 MB/s 
     - 668.6 MB 2.3 MB/s
     |████████████████████████████████| 5.8 MB 13.7 MB/s 
     |████████████████████████████████| 462 kB 69.7 MB/s 
     |████████████████████████████████| 1.4 MB 63.9 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.9.1
    Uninstalling tensorflow-2.9

In [4]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

### Prepare the dataset

Import your dataset. For example

-----> dataset.zip/train/[train_images]

-----> dataset.zip/validate/[validate_images]



In [5]:
!unzip -q dataset.zip

## Train the object detection model

### Step 1: Load the dataset

* Images in `train_data` is used to train the custom object detection model.
* Images in `val_data` is used to check if the model can generalize well to new images that it hasn't seen before.

In [6]:
train_data = object_detector.DataLoader.from_pascal_voc(
    'dataset/train',
    'dataset/train',
    ['mango','apple']
)

val_data = object_detector.DataLoader.from_pascal_voc(
    'dataset/validate',
    'dataset/validate',
    ['mango','apple']
)

### Step 2: Select a model architecture

EfficientDet-Lite[0-4] are a family of mobile/IoT-friendly object detection models derived from the [EfficientDet](https://arxiv.org/abs/1911.09070) architecture.

Here is the performance of each EfficientDet-Lite models compared to each others.

| Model architecture | Size(MB)* | Latency(ms)** | Average Precision*** |
|--------------------|-----------|---------------|----------------------|
| EfficientDet-Lite0 | 4.4       | 146           | 25.69%               |
| EfficientDet-Lite1 | 5.8       | 259           | 30.55%               |
| EfficientDet-Lite2 | 7.2       | 396           | 33.97%               |
| EfficientDet-Lite3 | 11.4      | 716           | 37.70%               |
| EfficientDet-Lite4 | 19.9      | 1886          | 41.96%               |

<i> * Size of the integer quantized models. <br/>
** Latency measured on Raspberry Pi 4 using 4 threads on CPU. <br/>
*** Average Precision is the mAP (mean Average Precision) on the COCO 2017 validation dataset.
</i>

In this notebook, we use EfficientDet-Lite0 to train our model. You can choose other model architectures depending on whether speed or accuracy is more important to you.

In [7]:
spec = model_spec.get('efficientdet_lite1')

### Step 3: Train the TensorFlow model with the training data.

* Set `epochs = 20`, which means it will go through the training dataset 20 times. You can look at the validation accuracy during training and stop when you see validation loss (`val_loss`) stop decreasing to avoid overfitting.
* Set `batch_size = 4` here so you will see that it takes 15 steps to go through the 62 images in the training dataset.
* Set `train_whole_model=True` to fine-tune the whole model instead of just training the head layer to improve accuracy. The trade-off is that it may take longer to train the model.

In [8]:
model = object_detector.create(train_data, model_spec=spec, batch_size=32, train_whole_model=True, epochs=200, validation_data=val_data)

Epoch 1/200
3/3 [==============================] - 58s 3s/step - det_loss: 1.7901 - cls_loss: 1.1465 - box_loss: 0.0129 - reg_l2_loss: 0.0706 - loss: 1.8607 - learning_rate: 0.0213 - gradient_norm: 0.8679
Epoch 2/200
3/3 [==============================] - 12s 5s/step - det_loss: 1.7103 - cls_loss: 1.1065 - box_loss: 0.0121 - reg_l2_loss: 0.0706 - loss: 1.7809 - learning_rate: 0.0400 - gradient_norm: 0.8951
Epoch 3/200
3/3 [==============================] - 15s 6s/step - det_loss: 1.5026 - cls_loss: 0.9994 - box_loss: 0.0101 - reg_l2_loss: 0.0706 - loss: 1.5732 - learning_rate: 0.0400 - gradient_norm: 1.0866
Epoch 4/200
3/3 [==============================] - 15s 5s/step - det_loss: 1.3066 - cls_loss: 0.7934 - box_loss: 0.0103 - reg_l2_loss: 0.0706 - loss: 1.3772 - learning_rate: 0.0400 - gradient_norm: 1.8279
Epoch 5/200
3/3 [==============================] - 15s 6s/step - det_loss: 1.0197 - cls_loss: 0.5993 - box_loss: 0.0084 - reg_l2_loss: 0.0706 - loss: 1.0904 - learning_rate: 0.0400

### Step 4. Evaluate the model with the validation data.

After training the object detection model using the images in the training dataset, use the 10 images in the validation dataset to evaluate how the model performs against new data it has never seen before.

The evaluation metrics are same as [COCO](https://cocodataset.org/#detection-eval).

In [9]:
model.evaluate(val_data)

1/1 [==============================] - 10s 10s/step



{'AP': 0.6235461,
 'AP50': 0.9063771,
 'AP75': 0.6050029,
 'AP_/mango': 0.6235461,
 'APl': 0.6086459,
 'APm': 0.82244223,
 'APs': -1.0,
 'ARl': 0.6675,
 'ARm': 0.8666667,
 'ARmax1': 0.47906977,
 'ARmax10': 0.63255817,
 'ARmax100': 0.68139535,
 'ARs': -1.0}

### Step 5: Export as a TensorFlow Lite model.

Export the trained object detection model to the TensorFlow Lite format by specifying which folder you want to export the quantized model to. The default post-training quantization technique is [full integer quantization](https://www.tensorflow.org/lite/performance/post_training_integer_quant). This allows the TensorFlow Lite model to be smaller, run faster on Raspberry Pi CPU and also compatible with the Google Coral EdgeTPU.

In [10]:
model.export(export_dir='.', tflite_filename='model.tflite')